In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# Base directory of your dataset
base_dir = '/kaggle/input/palm-health/Processed Dataset/RGB' 

In [ ]:
IMG_SIZE = (224, 224)  # You can adjust this size based on your needs

In [ ]:
def load_rgb_images(base_dir, img_size):
    classes = ['Healthy', 'Infected', 'Dead']
    images = []
    labels = []
    class_indices = {'Healthy': 0, 'Infected': 1, 'Dead': 2}
    
    for cls in classes:
        class_dir = os.path.join(base_dir, cls)
        if not os.path.exists(class_dir):
            print(f"Class directory not found: {class_dir}")
            continue

        palm_ids = [d for d in os.listdir(class_dir) if os.path.isdir(os.path.join(class_dir, d))]
        
        for palm_id in palm_ids:
            palm_dir = os.path.join(class_dir, palm_id)
            image_files = [f for f in os.listdir(palm_dir) if f.endswith('-A.png')]
            
            for img_file in image_files:
                img_path = os.path.join(palm_dir, img_file)
                # Load RGB image
                img = tf.keras.preprocessing.image.load_img(
                    img_path, target_size=img_size)
                img_array = tf.keras.preprocessing.image.img_to_array(img)
                images.append(img_array)
                labels.append(class_indices[cls])
    
    images = np.array(images) / 255.0  # Normalize images
    labels = np.array(labels)
    return images, labels

In [ ]:
# Load images and labels
images, labels = load_rgb_images(base_dir, IMG_SIZE)

print(f'Total images loaded: {len(images)}')

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    images, labels, test_size=0.2, stratify=labels, random_state=42)

print(f'Training samples: {len(X_train)}')
print(f'Validation samples: {len(X_val)}')

In [ ]:

import keras
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(32,kernel_size=(3,3),strides=2,padding='same',activation='relu',input_shape=IMG_SIZE))
model.add(keras.layers.MaxPool2D(pool_size=(2,2),strides=2,padding='same'))
model.add(keras.layers.Conv2D(64,kernel_size=(3,3),strides=2,activation='relu',padding='same'))
model.add(keras.layers.MaxPool2D((2,2),2,padding='same'))
model.add(keras.layers.Conv2D(128,kernel_size=(3,3),strides=2,activation='relu',padding='same'))
model.add(keras.layers.MaxPool2D((2,2),2,padding='same'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024,activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(3,activation='softmax'))
model.summary()

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Rescale pixel values
    rotation_range=30,           # Random rotation
    width_shift_range=0.2,       # Horizontal shift
    height_shift_range=0.2,      # Vertical shift
    shear_range=0.2,             # Shear transformations
    zoom_range=0.2,              # Zoom
    horizontal_flip=True,        # Randomly flip images
    fill_mode='nearest'          # Fill mode for new pixels
)

# Validation data generator (only rescaling)
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7)
model_checkpoint = ModelCheckpoint('best_palm_model.keras', monitor='val_accuracy', save_best_only=True)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

# Batch size
batch_size = 16

train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size, shuffle=True)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size, shuffle=False)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[model_checkpoint, reduce_lr]
)